In [1]:
from setuptools import find_packages

In [2]:
import torch

In [3]:
from torch import nn

In [2]:
find_packages(exclude=('configs', 'tools', 'demo'))

['mmcv_custom',
 'mmdet',
 'mmcv_custom.runner',
 'mmdet.apis',
 'mmdet.core',
 'mmdet.datasets',
 'mmdet.utils',
 'mmdet.models',
 'mmdet.core.visualization',
 'mmdet.core.post_processing',
 'mmdet.core.utils',
 'mmdet.core.mask',
 'mmdet.core.evaluation',
 'mmdet.core.anchor',
 'mmdet.core.export',
 'mmdet.core.bbox',
 'mmdet.core.bbox.match_costs',
 'mmdet.core.bbox.iou_calculators',
 'mmdet.core.bbox.assigners',
 'mmdet.core.bbox.samplers',
 'mmdet.core.bbox.coder',
 'mmdet.datasets.pipelines',
 'mmdet.datasets.api_wrappers',
 'mmdet.datasets.samplers',
 'mmdet.models.losses',
 'mmdet.models.dense_heads',
 'mmdet.models.necks',
 'mmdet.models.utils',
 'mmdet.models.backbones',
 'mmdet.models.detectors',
 'mmdet.models.roi_heads',
 'mmdet.models.roi_heads.shared_heads',
 'mmdet.models.roi_heads.roi_extractors',
 'mmdet.models.roi_heads.bbox_heads',
 'mmdet.models.roi_heads.mask_heads']

In [3]:
!pwd

/Users/yupeng/Documents/2021summer/SummerIntern/QueryInst


In [4]:
find_packages(exclude=('configs', 'tools'))  

['mmcv_custom',
 'mmdet',
 'mmcv_custom.runner',
 'mmdet.apis',
 'mmdet.core',
 'mmdet.datasets',
 'mmdet.utils',
 'mmdet.models',
 'mmdet.core.visualization',
 'mmdet.core.post_processing',
 'mmdet.core.utils',
 'mmdet.core.mask',
 'mmdet.core.evaluation',
 'mmdet.core.anchor',
 'mmdet.core.export',
 'mmdet.core.bbox',
 'mmdet.core.bbox.match_costs',
 'mmdet.core.bbox.iou_calculators',
 'mmdet.core.bbox.assigners',
 'mmdet.core.bbox.samplers',
 'mmdet.core.bbox.coder',
 'mmdet.datasets.pipelines',
 'mmdet.datasets.api_wrappers',
 'mmdet.datasets.samplers',
 'mmdet.models.losses',
 'mmdet.models.dense_heads',
 'mmdet.models.necks',
 'mmdet.models.utils',
 'mmdet.models.backbones',
 'mmdet.models.detectors',
 'mmdet.models.roi_heads',
 'mmdet.models.roi_heads.shared_heads',
 'mmdet.models.roi_heads.roi_extractors',
 'mmdet.models.roi_heads.bbox_heads',
 'mmdet.models.roi_heads.mask_heads']

In [6]:
def parse_requirements(fname='requirements.txt', with_version=True):
    """Parse the package dependencies listed in a requirements file but strips
    specific versioning information.

    Args:
        fname (str): path to requirements file
        with_version (bool, default=False): if True include version specs

    Returns:
        List[str]: list of requirements items

    CommandLine:
        python -c "import setup; print(setup.parse_requirements())"
    """
    import sys
    from os.path import exists
    import re
    require_fpath = fname

    def parse_line(line):
        """Parse information from a line in a requirements text file."""
        if line.startswith('-r '):
            # Allow specifying requirements in other files
            target = line.split(' ')[1]
            for info in parse_require_file(target):
                yield info
        else:
            info = {'line': line}
            if line.startswith('-e '):
                info['package'] = line.split('#egg=')[1]
            elif '@git+' in line:
                info['package'] = line
            else:
                # Remove versioning from the package
                pat = '(' + '|'.join(['>=', '==', '>']) + ')'
                parts = re.split(pat, line, maxsplit=1)
                parts = [p.strip() for p in parts]

                info['package'] = parts[0]
                if len(parts) > 1:
                    op, rest = parts[1:]
                    if ';' in rest:
                        # Handle platform specific dependencies
                        # http://setuptools.readthedocs.io/en/latest/setuptools.html#declaring-platform-specific-dependencies
                        version, platform_deps = map(str.strip,
                                                     rest.split(';'))
                        info['platform_deps'] = platform_deps
                    else:
                        version = rest  # NOQA
                    info['version'] = (op, version)
            yield info

    def parse_require_file(fpath):
        with open(fpath, 'r') as f:
            for line in f.readlines():
                line = line.strip()
                if line and not line.startswith('#'):
                    for info in parse_line(line):
                        yield info

    def gen_packages_items():
        if exists(require_fpath):
            for info in parse_require_file(require_fpath):
                parts = [info['package']]
                if with_version and 'version' in info:
                    parts.extend(info['version'])
                if not sys.version.startswith('3.4'):
                    # apparently package_deps are broken in 3.4
                    platform_deps = info.get('platform_deps')
                    if platform_deps is not None:
                        parts.append(';' + platform_deps)
                item = ''.join(parts)
                yield item

    packages = list(gen_packages_items())
    return packages

In [7]:
parse_requirements('requirements/build.txt')

['cython', 'numpy']

In [8]:
parse_requirements('requirements/tests.txt')

['asynctest',
 'codecov',
 'flake8',
 'interrogate',
 'isort==4.3.21',
 'kwarray',
 'onnx==1.7.0',
 'onnxruntime==1.5.1',
 'pytest',
 'ubelt',
 'xdoctest>=0.10.0',
 'yapf']

In [10]:
parse_requirements('requirements/runtime.txt'),

(['matplotlib',
  'numpy',
  'pycocotools; platform_system=="Linux"',
  'pycocotools-windows; platform_system=="Windows"',
  'six',
  'terminaltables',
  'timm'],)

In [13]:
init_proposal_bboxes = nn.Embedding(10, 4)
res = nn.init.constant_(init_proposal_bboxes.weight[:, :2], 0.5)
res

tensor([[0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000]], grad_fn=<AsStridedBackward>)

In [19]:
nn.init.constant_(init_proposal_bboxes.weight[:, 2:], 1)

tensor([[1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]], grad_fn=<AsStridedBackward>)

In [20]:
init_proposal_bboxes.weight

Parameter containing:
tensor([[0.5000, 0.5000, 1.0000, 1.0000],
        [0.5000, 0.5000, 1.0000, 1.0000],
        [0.5000, 0.5000, 1.0000, 1.0000],
        [0.5000, 0.5000, 1.0000, 1.0000],
        [0.5000, 0.5000, 1.0000, 1.0000],
        [0.5000, 0.5000, 1.0000, 1.0000],
        [0.5000, 0.5000, 1.0000, 1.0000],
        [0.5000, 0.5000, 1.0000, 1.0000],
        [0.5000, 0.5000, 1.0000, 1.0000],
        [0.5000, 0.5000, 1.0000, 1.0000]], requires_grad=True)

In [23]:
a = torch.zeros((2, 10, 4))
a.shape

torch.Size([2, 10, 4])

In [24]:
len(a)

2

In [26]:
b = [a[i] for i in range(len(a))]
b

[tensor([[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]),
 tensor([[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]])]

In [4]:
def bbox2roi(bbox_list):
    """Convert a list of bboxes to roi format.

    Args:
        bbox_list (list[Tensor]): a list of bboxes corresponding to a batch
            of images.

    Returns:
        Tensor: shape (n, 5), [batch_ind, x1, y1, x2, y2]
    """
    rois_list = []
    for img_id, bboxes in enumerate(bbox_list):
        # bboxes.shape: (num_proposals, 4)
        if bboxes.size(0) > 0:
            img_inds = bboxes.new_full((bboxes.size(0), 1), img_id)
            rois = torch.cat([img_inds, bboxes[:, :4]], dim=-1)
        else:
            rois = bboxes.new_zeros((0, 5))
        rois_list.append(rois)
    rois = torch.cat(rois_list, 0)
    return rois

In [33]:
c = torch.rand((4, 2, 4))
d = bbox2roi(c)
d

tensor([[0.0000, 0.4140, 0.7233, 0.4780, 0.1765],
        [0.0000, 0.8594, 0.4138, 0.2550, 0.2919],
        [1.0000, 0.1803, 0.6177, 0.8708, 0.6756],
        [1.0000, 0.6536, 0.2452, 0.3715, 0.2674],
        [2.0000, 0.1102, 0.4175, 0.5090, 0.7105],
        [2.0000, 0.4338, 0.6390, 0.5860, 0.4554],
        [3.0000, 0.8406, 0.4782, 0.0737, 0.2856],
        [3.0000, 0.0529, 0.1080, 0.3798, 0.3995]])

In [31]:
d.shape

torch.Size([8, 5])

In [34]:
c

tensor([[[0.4140, 0.7233, 0.4780, 0.1765],
         [0.8594, 0.4138, 0.2550, 0.2919]],

        [[0.1803, 0.6177, 0.8708, 0.6756],
         [0.6536, 0.2452, 0.3715, 0.2674]],

        [[0.1102, 0.4175, 0.5090, 0.7105],
         [0.4338, 0.6390, 0.5860, 0.4554]],

        [[0.8406, 0.4782, 0.0737, 0.2856],
         [0.0529, 0.1080, 0.3798, 0.3995]]])

In [16]:
proposal_boxes = torch.rand(4, 2, 4)
proposal_list = [proposal_boxes[i] for i in range(len(proposal_boxes))]
rois = bbox2roi(proposal_list)

In [17]:
rois

tensor([[0.0000, 0.8674, 0.6879, 0.2520, 0.3844],
        [0.0000, 0.0416, 0.9977, 0.4560, 0.3463],
        [1.0000, 0.2048, 0.4977, 0.6467, 0.9364],
        [1.0000, 0.1216, 0.4102, 0.1659, 0.1015],
        [2.0000, 0.2531, 0.7194, 0.8648, 0.3340],
        [2.0000, 0.2727, 0.6682, 0.8327, 0.1480],
        [3.0000, 0.5849, 0.6169, 0.1098, 0.8604],
        [3.0000, 0.8024, 0.9218, 0.0160, 0.8042]])

In [5]:
in_channels = [2, 3, 5, 7]
scales = [340, 170, 84, 43]
inputs = [torch.rand(1, c, s, s) for c, s in zip(in_channels, scales)]
inputs[0].shape

torch.Size([1, 2, 340, 340])

In [6]:
inputs[1].shape

torch.Size([1, 3, 170, 170])

In [8]:
# imgs_whwh.append(imgs[0].new_tensor([[w, h, w, h]]))
img_0 = torch.rand((2, 3, 3, 3))
img_0

tensor([[[[0.1859, 0.4968, 0.5601],
          [0.1935, 0.3251, 0.7271],
          [0.2630, 0.7858, 0.9863]],

         [[0.4258, 0.5859, 0.3402],
          [0.0410, 0.3063, 0.6568],
          [0.9656, 0.2459, 0.9027]],

         [[0.5308, 0.5463, 0.0868],
          [0.5445, 0.8437, 0.9497],
          [0.3916, 0.1916, 0.3441]]],


        [[[0.0472, 0.4806, 0.7984],
          [0.2977, 0.9824, 0.7561],
          [0.5655, 0.2376, 0.6125]],

         [[0.8725, 0.2155, 0.0750],
          [0.0554, 0.6008, 0.6296],
          [0.3653, 0.3989, 0.5753]],

         [[0.5374, 0.4930, 0.4973],
          [0.5730, 0.6311, 0.8633],
          [0.9826, 0.0937, 0.3937]]]])

In [10]:
img_0.new_tensor([[2, 2, 2, 2]])

tensor([[2., 2., 2., 2.]])

In [13]:
imgs_whwh = torch.rand((2, 1, 4))
proposals = torch.rand((3, 4))
proposals

tensor([[0.1005, 0.6047, 0.4954, 0.0234],
        [0.5817, 0.8683, 0.5542, 0.7398],
        [0.0536, 0.0303, 0.5816, 0.9956]])

In [14]:
imgs_whwh

tensor([[[0.5256, 0.1174, 0.3877, 0.4371]],

        [[0.2512, 0.5584, 0.2768, 0.0615]]])

In [16]:
final_proposals = proposals * imgs_whwh

In [17]:
final_proposals.shape

torch.Size([2, 3, 4])

In [18]:
final_proposals

tensor([[[0.0528, 0.0710, 0.1921, 0.0102],
         [0.3057, 0.1020, 0.2149, 0.3234],
         [0.0281, 0.0036, 0.2255, 0.4352]],

        [[0.0253, 0.3377, 0.1372, 0.0014],
         [0.1461, 0.4848, 0.1534, 0.0455],
         [0.0134, 0.0169, 0.1610, 0.0612]]])

In [22]:
num_proposals = 3
proposal_feature_channel = 3
init_proposal_features = nn.Embedding(num_proposals, proposal_feature_channel)
init_proposal_features.weight

Parameter containing:
tensor([[-0.4601,  1.9385, -1.3533],
        [-0.0058, -1.2428,  0.0770],
        [-0.5888, -0.8354, -1.3720]], requires_grad=True)

In [25]:
init_proposal_features_weight = init_proposal_features.weight.clone()
init_proposal_features_weight

tensor([[-0.4601,  1.9385, -1.3533],
        [-0.0058, -1.2428,  0.0770],
        [-0.5888, -0.8354, -1.3720]], grad_fn=<CloneBackward>)

In [26]:
init_proposal_features_weight[None]

tensor([[[-0.4601,  1.9385, -1.3533],
         [-0.0058, -1.2428,  0.0770],
         [-0.5888, -0.8354, -1.3720]]], grad_fn=<UnsqueezeBackward0>)

In [27]:
init_proposal_features_weight[]

SyntaxError: invalid syntax (<ipython-input-27-bdf5fa19dc76>, line 1)

In [28]:
init_proposal_features_weight[1]

tensor([-0.0058, -1.2428,  0.0770], grad_fn=<SelectBackward>)

In [29]:
init_proposal_features_weight.size()

torch.Size([3, 3])

In [30]:
*init_proposal_features_weight.size()

SyntaxError: can't use starred expression here (<ipython-input-30-8e71f608340f>, line 1)

In [31]:
num_imgs = 2
init_proposal_features_weight[None].expand(num_imgs, *init_proposal_features_weight.size())

tensor([[[-0.4601,  1.9385, -1.3533],
         [-0.0058, -1.2428,  0.0770],
         [-0.5888, -0.8354, -1.3720]],

        [[-0.4601,  1.9385, -1.3533],
         [-0.0058, -1.2428,  0.0770],
         [-0.5888, -0.8354, -1.3720]]], grad_fn=<ExpandBackward>)

In [32]:
init_proposal_features_weight[None].expand(num_imgs, *init_proposal_features_weight.size()).size()

torch.Size([2, 3, 3])

In [7]:
featmap_strides = [4, 8, 16, 32]
len(featmap_strides)

4

In [42]:
def map_roi_levels(rois, num_levels):
    """Map rois to corresponding feature levels by scales.

    - scale < finest_scale * 2: level 0
    - finest_scale * 2 <= scale < finest_scale * 4: level 1
    - finest_scale * 4 <= scale < finest_scale * 8: level 2
    - scale >= finest_scale * 8: level 3

    Args:
        rois (Tensor): Input RoIs, shape (k, 5).
        num_levels (int): Total level number.

    Returns:
        Tensor: Level index (0-based) of each RoI, shape (k, )
    """
    # rois: (batch * num_proposals, 5)
    scale = torch.sqrt(
        (rois[:, 3] - rois[:, 1]) * (rois[:, 4] - rois[:, 2]))
    print(scale)
    target_lvls = torch.floor(torch.log2(scale / 56 + 1e-6))
    print(target_lvls)
    target_lvls = target_lvls.clamp(min=0, max=num_levels - 1).long()
    return target_lvls

In [18]:
rois

tensor([[0.0000, 0.8674, 0.6879, 0.2520, 0.3844],
        [0.0000, 0.0416, 0.9977, 0.4560, 0.3463],
        [1.0000, 0.2048, 0.4977, 0.6467, 0.9364],
        [1.0000, 0.1216, 0.4102, 0.1659, 0.1015],
        [2.0000, 0.2531, 0.7194, 0.8648, 0.3340],
        [2.0000, 0.2727, 0.6682, 0.8327, 0.1480],
        [3.0000, 0.5849, 0.6169, 0.1098, 0.8604],
        [3.0000, 0.8024, 0.9218, 0.0160, 0.8042]])

In [43]:
map_roi_res = map_roi_levels(rois, 4)

tensor([0.4322,    nan, 0.4403,    nan,    nan,    nan,    nan, 0.3041])
tensor([-8., nan, -7., nan, nan, nan, nan, -8.])


In [44]:
map_roi_res.shape

torch.Size([8])

In [45]:
map_roi_res

tensor([                   0, -9223372036854775808,                    0,
        -9223372036854775808, -9223372036854775808, -9223372036854775808,
        -9223372036854775808,                    0])

In [32]:
map_roi_res == 1

tensor([False, False, False, False, False, False, False, False])

In [34]:
all_one = torch.ones((4, 2, 4))
all_one_res = bbox2roi(all_one)
all_one_res

tensor([[0., 1., 1., 1., 1.],
        [0., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [2., 1., 1., 1., 1.],
        [2., 1., 1., 1., 1.],
        [3., 1., 1., 1., 1.],
        [3., 1., 1., 1., 1.]])

In [36]:
map_roi_levels(all_one_res, 4)

tensor([0., 0., 0., 0., 0., 0., 0., 0.])


tensor([0, 0, 0, 0, 0, 0, 0, 0])

In [37]:
torch.floor(torch.rand(3))

tensor([0., 0., 0.])

In [38]:
torch.floor(torch.ones(3))

tensor([1., 1., 1.])

In [46]:
test_roi = bbox2roi(torch.rand((4, 2, 4)))
test_roi

tensor([[0.0000, 0.7629, 0.3841, 0.2630, 0.7933],
        [0.0000, 0.7373, 0.4391, 0.9600, 0.6475],
        [1.0000, 0.6860, 0.6027, 0.0100, 0.0610],
        [1.0000, 0.1370, 0.0319, 0.7754, 0.8904],
        [2.0000, 0.2996, 0.0309, 0.0516, 0.9780],
        [2.0000, 0.2541, 0.1225, 0.8034, 0.4431],
        [3.0000, 0.9790, 0.4394, 0.4774, 0.2575],
        [3.0000, 0.1794, 0.8235, 0.9170, 0.6085]])

In [47]:
torch.sqrt((test_roi[:, 3] - test_roi[:, 1]) * (test_roi[:, 4] - test_roi[:, 2]))

tensor([   nan, 0.2154, 0.6051, 0.7404,    nan, 0.4196, 0.3020,    nan])

In [48]:
(test_roi[:, 3] - test_roi[:, 1]) * (test_roi[:, 4] - test_roi[:, 2])

tensor([-0.2045,  0.0464,  0.3662,  0.5481, -0.2349,  0.1761,  0.0912, -0.1586])

In [49]:
test_roi[:, 3] - test_roi[:, 1]

tensor([-0.4999,  0.2227, -0.6760,  0.6384, -0.2480,  0.5494, -0.5015,  0.7376])

In [50]:
test_roi[:, 3]

tensor([0.2630, 0.9600, 0.0100, 0.7754, 0.0516, 0.8034, 0.4774, 0.9170])

In [57]:
mask = map_roi_res == 0
inds = mask.nonzero(as_tuple=False).squeeze(1)
inds

tensor([0, 2, 7])

In [59]:
mask

tensor([ True, False,  True, False, False, False, False,  True])

In [60]:
map_roi_res

tensor([                   0, -9223372036854775808,                    0,
        -9223372036854775808, -9223372036854775808, -9223372036854775808,
        -9223372036854775808,                    0])

In [58]:
mask.nonzero(as_tuple=False)

tensor([[0],
        [2],
        [7]])

In [61]:
test_roi

tensor([[0.0000, 0.7629, 0.3841, 0.2630, 0.7933],
        [0.0000, 0.7373, 0.4391, 0.9600, 0.6475],
        [1.0000, 0.6860, 0.6027, 0.0100, 0.0610],
        [1.0000, 0.1370, 0.0319, 0.7754, 0.8904],
        [2.0000, 0.2996, 0.0309, 0.0516, 0.9780],
        [2.0000, 0.2541, 0.1225, 0.8034, 0.4431],
        [3.0000, 0.9790, 0.4394, 0.4774, 0.2575],
        [3.0000, 0.1794, 0.8235, 0.9170, 0.6085]])

In [62]:
test_roi[inds]

tensor([[0.0000, 0.7629, 0.3841, 0.2630, 0.7933],
        [1.0000, 0.6860, 0.6027, 0.0100, 0.0610],
        [3.0000, 0.1794, 0.8235, 0.9170, 0.6085]])

In [63]:
inds

tensor([0, 2, 7])

In [64]:
proposal_feat = torch.rand((2, 2, 3))
proposal_feat

tensor([[[0.6784, 0.7932, 0.5145],
         [0.0992, 0.0859, 0.5313]],

        [[0.4867, 0.9737, 0.1892],
         [0.3729, 0.2131, 0.0048]]])

In [65]:
proposal_feat.permute(1, 0, 2)

tensor([[[0.6784, 0.7932, 0.5145],
         [0.4867, 0.9737, 0.1892]],

        [[0.0992, 0.0859, 0.5313],
         [0.3729, 0.2131, 0.0048]]])

In [4]:
test_new_zeros = torch.rand((2, 3, 4))
test_new_zeros_1 = torch.rand((2, 4))
gen = test_new_zeros.new_zeros(len(test_new_zeros))
gen_1 = test_new_zeros_1.new_zeros(len(test_new_zeros_1))

In [5]:
gen.shape

torch.Size([2])

In [7]:
gen_1.shape

torch.Size([2])

In [8]:
test_new_zeros_1[0]

tensor([0.8472, 0.7459, 0.4239, 0.9840])

In [9]:
test_new_zeros.shape

torch.Size([2, 3, 4])

In [10]:
test_new_zeros_1 * 4

tensor([[3.3887, 2.9836, 1.6955, 3.9359],
        [2.0148, 0.6900, 0.7875, 2.2636]])

In [12]:
label = torch.rand(3)
inds = torch.stack((label, label + 1, label + 2, label + 3), 1)
inds.shape

torch.Size([3, 4])

In [13]:
label

tensor([0.0178, 0.2046, 0.7053])

In [14]:
inds

tensor([[0.0178, 1.0178, 2.0178, 3.0178],
        [0.2046, 1.2046, 2.2046, 3.2046],
        [0.7053, 1.7053, 2.7053, 3.7053]])

In [16]:
num_rois = 1
num_proposals = 3
pos_is_gts_ = torch.ones(num_proposals)
pos_keep = 1 - pos_is_gts_  # (num_proposals,) 实际上全是1
keep_inds = pos_is_gts_.new_ones(num_rois)  # (num_rois, )
keep_inds[:len(pos_is_gts_)] = pos_keep

RuntimeError: The expanded size of the tensor (1) must match the existing size (3) at non-singleton dimension 0.  Target sizes: [1].  Tensor sizes: [3]